Try for 100 random articles from...
- all the articles
- GDELT
- etc. 

In [9]:
from gzip import GzipFile
import pandas as pd
import os

with GzipFile('data/GDELT_VGKG/vgkg-20160427-part1.csv.gz') as gzipfile:
    df = pd.read_csv(gzipfile, nrows=100)
    
df.head()

,DATE,DocumentIdentifier,ImageURL,RawJSON
0,20160307200000,http://sputniknews.com/us/20160307/1035926176/...,http://cdn2.img.sputniknews.com/images/103268/...,"{ ""responses"": [ { ""faceAnnotations"": [ { ""bou..."
1,20160318080000,http://sverigesradio.se/sida/artikel.aspx?prog...,http://sverigesradio.se/sida/images/1646/36601...,"{ ""responses"": [ { ""labelAnnotations"": [ { ""mi..."
2,20160318080000,http://www.ht.no/nyheter/2016/03/18/Legger-seg...,http://www.ht.no/incoming/article12288611.ece/...,"{ ""responses"": [ { ""labelAnnotations"": [ { ""mi..."
3,20160423200000,http://www.arabianewspaper.com/malayalam-news/...,http://www.arabianewspaper.com/media/selfie-pn...,"{ ""responses"": [ { ""labelAnnotations"": [ { ""mi..."
4,20160322080000,http://www.cronica.com.mx/notas/2016/951571.html,http://www.cronica.com.mx/nimagenes/4/2016-03-...,"{ ""responses"": [ { ""labelAnnotations"": [ { ""mi..."


In [10]:
import urllib
import json
responses = list()
for index, (data, document_identifier, image_url, raw_json) in df.iterrows():
    url = 'https://getvideo.p.mashape.com/?' + urllib.parse.urlencode({"url":document_identifier})
    req = urllib.request.Request(url)
    print(urllib.request.Request(document_identifier).host)
    req.add_header('X-Mashape-Key', os.environ["MASHAPE_KEY"])
    try:
        response = json.loads(urllib.request.urlopen(req).read())
    except e:
        response = {"status":False,"message":"Crawler caught Exception: %s" % e}
    responses.append(response)

sputniknews.com
sverigesradio.se
www.ht.no
www.arabianewspaper.com
www.cronica.com.mx
www.alshahedkw.com
www.timesunion.com
deportes.infonews.com
www.centralwesterndaily.com.au
www.jornaldamidia.com.br
www.t-online.de
newsok.com
www.sydsvenskan.se
www.haz.de
www.paris-normandie.fr
www.napi.hu
www.sltrib.com
english.cntv.cn
www.auraavis.no
congan.com.vn
www.teknologadam.com
www.omroepbrabant.nl
www.alquds.com
russian.rt.com
stirileprotv.ro
afamily.vn
www.romanialibera.ro
www.dailymail.co.uk
www.dailymail.co.uk
www.folkebladet.no
el-carabobeno.com
www.szon.hu
www.thenational.scot
aaj.tv
bilbaoya.com
www.theguardian.com
www.sankei.com
myslo.ru
vietbao.vn
suracapulco.mx
ladiaria.com.uy
economictimes.indiatimes.com
oglobo.globo.com
www.pressdemocrat.com
nuovavenezia.gelocal.it
www.springnews.co.th
www.grandain.com
www.mathrubhumi.com
chuhal.mn
blog.udn.com
notizie.tiscali.it
www.al-fadjr.com


HTTPError: HTTP Error 504: Gateway Time-out

In [93]:
len(responses)

51

In [52]:
# write the result to a file. Name format is <key of first row>-<key of last row>
import pickle
result_path = "data/GDELT_VGKG/preprocessed/video_json/%s-%s" % (int(df.iloc[[0]].DATE), int(df.iloc[[len(responses) - 1]].DATE))
file = open(result_path, "wb+")
pickle.dump(responses, file) # Date is unique(?) TODO find out

In [53]:
result_path

'data/GDELT_VGKG/preprocessed/video_json/20160307200000-20160404160000'

In [95]:
str(int(df.iloc[[0]].DATE))

'20160307200000'

In [55]:
for response in responses:
    print(len(response.keys()), end=",")

2,2,2,2,2,2,2,2,2,2,6,2,2,2,2,2,2,13,2,6,2,2,2,2,2,2,2,6,2,2,6,2,2,2,2,2,2,2,2,2,2,2,6,2,2,2,2,2,2,2,2,

In [99]:
interesting_responses = [response for response in responses if response["status"]]
len(interesting_responses)
# 6/51 (more than 10%!) have videos embedded

6

In [89]:
RenderJSON(interesting_responses[5])

In [103]:
failed_sites = dict()
for index in range(len(responses)):
    key = urllib.request.Request(df.iloc[[index]].DocumentIdentifier).host
    print(df.iloc[index].DocumentIdentifier, end=": ")
    if not responses[index]["status"]:
        print("Failed")
        if not key in failed_sites:
            failed_sites[key] = 1
        else:
            failed_sites[key]+=1
    else:
        print("success")
        

http://sputniknews.com/us/20160307/1035926176/trump-ties-racist-mobster.html: Failed
http://sverigesradio.se/sida/artikel.aspx?programid=1646&artikel=6392188: Failed
http://www.ht.no/nyheter/2016/03/18/Legger-seg-flat-etter-at-Alf-fant-denne-gjenstanden-i-maten-12320366.ece: Failed
http://www.arabianewspaper.com/malayalam-news/read/13990/narrow-escape-for-youth-who-tried-selfie-with-temple-tusker/.B-N: Failed
http://www.cronica.com.mx/notas/2016/951571.html: Failed
http://www.alshahedkw.com/index.php?option=com_content&view=article&id=148662:2016-03-15-17-33-23&catid=481:09&Itemid=551: Failed
http://www.timesunion.com/entertainment/celebrities/article/Here-are-some-of-the-hottest-models-over-40-7224564.php: Failed
http://deportes.infonews.com/nota/286770/las-cinco-casacas-mas-polemicas-de-los: Failed
http://www.centralwesterndaily.com.au/story/3836753/police-crack-down-on-outlaw-motorcyle-clubs-in-orange-and-the-central-west/?cs=4076: Failed
http://www.jornaldamidia.com.br/2016/02/14/k

In [98]:
failed_sites

{'sputniknews.com': 1,
 'sverigesradio.se': 1,
 'www.ht.no': 1,
 'www.arabianewspaper.com': 1,
 'www.cronica.com.mx': 1,
 'www.alshahedkw.com': 1,
 'www.timesunion.com': 1,
 'deportes.infonews.com': 1,
 'www.centralwesterndaily.com.au': 1,
 'www.jornaldamidia.com.br': 1,
 'newsok.com': 1,
 'www.sydsvenskan.se': 1,
 'www.haz.de': 1,
 'www.paris-normandie.fr': 1,
 'www.napi.hu': 1,
 'www.sltrib.com': 1,
 'www.auraavis.no': 1,
 'www.teknologadam.com': 1,
 'www.omroepbrabant.nl': 1,
 'www.alquds.com': 1,
 'russian.rt.com': 1,
 'stirileprotv.ro': 1,
 'afamily.vn': 1,
 'www.romanialibera.ro': 1,
 'www.dailymail.co.uk': 1,
 'www.folkebladet.no': 1,
 'www.szon.hu': 1,
 'www.thenational.scot': 1,
 'aaj.tv': 1,
 'bilbaoya.com': 1,
 'www.theguardian.com': 1,
 'www.sankei.com': 1,
 'myslo.ru': 1,
 'vietbao.vn': 1,
 'suracapulco.mx': 1,
 'ladiaria.com.uy': 1,
 'economictimes.indiatimes.com': 1,
 'www.pressdemocrat.com': 1,
 'nuovavenezia.gelocal.it': 1,
 'www.springnews.co.th': 1,
 'www.grandain.co

## Download the Videos from the URL

In [112]:
# For some videos, only a playlist URL is provided. Retrieve the video URL for those.
playlist_resolution = dict()
for response in responses:
    if response["status"]:
        if response["is_playlist"]:
            for playlist_item in response["playlist_items"]:
                print("Retrieving Info for %s" % playlist_item)
                url = 'https://getvideo.p.mashape.com/?' + urllib.parse.urlencode({"url":playlist_item})
                req = urllib.request.Request(url)
                req.add_header('X-Mashape-Key', os.environ["MASHAPE_KEY"])
                try:
                    playlist_resolution[playlist_item] = json.loads(urllib.request.urlopen(req).read())
                except e:
                    playlist_resolution[playlist_item] = {"status":False,"message":"Crawler caught Exception: %s" % e}

Retrieving Info for https://www.youtube.com/embed/Qh9EwNurawE
Retrieving Info for https://www.youtube.com/embed/euJKmWnscJM?enablejsapi=1&origin=https://www.el-carabobeno.com&autoplay=0&cc_load_policy=0&iv_load_policy=1&loop=0&modestbranding=0&rel=1&showinfo=1&fs=1&theme=dark&color=red&autohide=2&controls=2&playsinline=0&


In [111]:
playlist_resolution.keys()

dict_keys(['https://www.youtube.com/embed/Qh9EwNurawE', 'https://www.youtube.com/embed/euJKmWnscJM?enablejsapi=1&origin=https://www.el-carabobeno.com&autoplay=0&cc_load_policy=0&iv_load_policy=1&loop=0&modestbranding=0&rel=1&showinfo=1&fs=1&theme=dark&color=red&autohide=2&controls=2&playsinline=0&'])

In [120]:
def dl_stream(url):
    print(url)

def get_image_url():
    return

def get_video_url():
    return

def get_id():
    return

def get_article_url():
    return


# TODO handle multiple streams (if they're not just the same video in multiple resolutions)
# TODO handle playlist with multiple videos
complete_results = list()
for index, response in zip(range(len(responses)), responses):
    if response["status"]:
        article_url = df.iloc[index].DocumentIdentifier
        image_url = df.iloc[index].ImageURL
        
        if response["is_playlist"]:
            if(len(response["playlist_items"]) > 0):
                video_url = playlist_resolution[response["playlist_items"][0]]["streams"][0]["url"]
                complete_results.append((article_url, image_url, video_url))
        else:
            video_url = response["streams"][0]["url"]
            complete_results.append((article_url, image_url, video_url))

In [122]:
from IPython.core.display import display, HTML

index = 0
for a_url, i_url, v_url in complete_results:
    display(HTML("""%d: <a href="%s">Article</a> <a href="%s">Image</a> <a href="%s">Video</a>""" 
                 % (index, a_url, i_url, v_url)))
    index+=1